In [ ]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)
import csv
from sklearn.preprocessing import MinMaxScaler

class STOCK_RNN(nn.Module):

  def __init__(self, config):
    
    super(STOCK_RNN, self).__init__()

    self.input_size = config["input_size"]
    self.hidden_size = config["hidden_size"]
    self.output_size = config["output_size"]
    self.num_layers = config["num_layers"]
    self.batch_size = config["batch_size"]

    # LSTM 설계
    self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, bidirectional=False, batch_first=True)
    # 출력층 설계
    self.linear = nn.Linear(self.hidden_size,self.output_size)

  def forward(self, input_features):

    # LSTM 리턴 = output (배치, 시퀀스, 은닉 상태), (hidden_state, cell_state)
    x, (h_n, c_n) = self.lstm(input_features)

    # output에서 마지막 시퀀스의 (배치, 은닉 상태) 정보를 가져옴
    h_t = x[:,-1,:]

    # 출력층: (배치, 출력)
    hypothesis = self.linear(h_t)
 
    return hypothesis

In [ ]:
# 데이터 읽기 함수
def load_dataset(fname):

  f = open(fname, 'r', encoding='cp949')

  # CSV 파일 읽기
  data = csv.reader(f,delimiter=',')

  # 헤더 건너뛰기
  next(data)

  data_X = []
  data_Y = []

  for row in data:
    # 오픈, 고가, 저가, 거래량 -> 숫자 변환
    data_X.append([float(i) for i in row[2:]])
    # 종가 -> 숫자 변환
    data_Y.append(float(row[1]))

  # MinMax 정규화 (예측하려는 종가 제외)
  scaler = MinMaxScaler()
  scaler.fit(data_X)
  data_X = scaler.transform(data_X)

  data_num = len(data_X)
  sequence_len = config["sequence_len"]
  seq_data_X, seq_data_Y = [], []

  # 윈도우 크기만큼 슬라이딩 하면서 데이터 생성
  for i in range(data_num-sequence_len):
    window_size = i+sequence_len
    seq_data_X.append(data_X[i:window_size])
    seq_data_Y.append([data_Y[window_size-1]])

  (train_X, train_Y) = (np.array(seq_data_X[:]),np.array(seq_data_Y[:]))
  train_X = torch.tensor(train_X, dtype=torch.float) 
  train_Y = torch.tensor(train_Y, dtype=torch.float) 

  print(train_X.shape) # (73,3,4)
  print(train_Y.shape) # (73,1)
  
  return (train_X, train_Y)

In [ ]:
# 모델 평가 결과 계산을 위해 텐서를 리스트로 변환하는 함수
def tensor2list(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()

# 평가 수행 함수
def do_test(model, test_dataloader):

  # 평가 모드 셋팅
  model.eval()

  # Batch 별로 예측값과 정답을 저장할 리스트 초기화
  predicts, golds = [], []
  
  with torch.no_grad():

    for step, batch in enumerate(test_dataloader):
  
      # .cuda()를 통해 메모리에 업로드
      batch = tuple(t.cuda() for t in batch)

      input_features, labels = batch
      hypothesis = model(input_features)

      x = tensor2list(hypothesis[:,0])
      y = tensor2list(labels)

      # 예측값과 정답을 리스트에 추가
      predicts.extend(x)
      golds.extend(y)
    
    # 소숫점 이하 1자리로 변환
    predicts = [round(i,1) for i in predicts]
    golds = [round(i[0],1) for i in golds]

    print("PRED=",predicts)
    print("GOLD=",golds)

# 모델 평가 함수
def test(config):

  model = STOCK_RNN(config).cuda()

  # 저장된 모델 가중치 로드
  model.load_state_dict(torch.load(os.path.join(config["output_dir"], config["model_name"])))

  # 데이터 load
  (features, labels) = load_dataset(confing["file_name"])
  
  test_features = TensorDataset(features, labels)
  test_dataloader = DataLoader(test_features, shuffle=True, batch_size=config["batch_size"])
  
  do_test(model, test_dataloader)

In [ ]:
# 모델 학습 함수
def train(config):

  # 모델 생성
  model = STOCK_RNN(config).cuda()

  # 데이터 읽기
  (input_features, labels) = load_dataset(config["file_name"])
  
  # TensorDataset/DataLoader를 통해 배치(batch) 단위로 데이터를 나누고 셔플(shuffle)
  train_features = TensorDataset(input_features, labels)
  train_dataloader = DataLoader(train_features, shuffle=True, batch_size=config["batch_size"])

  # MSE (Mean Square Error) 비용 함수 
  loss_func = nn.MSELoss()
  # 옵티마이저 함수 (역전파 알고리즘을 수행할 함수)
  optimizer = torch.optim.Adam(model.parameters(), lr=config["learn_rate"])

  for epoch in range(config["epoch"]+1):

    # 학습 모드 셋팅
    model.train()

    # epoch 마다 평균 비용을 저장하기 위한 리스트
    costs = []

    for (step, batch) in enumerate(train_dataloader):

      # batch = (input_features[step], labels[step])*batch_size
      # .cuda()를 통해 메모리에 업로드
      batch = tuple(t.cuda() for t in batch)

      # 각 feature 저장
      input_features, labels = batch

      # 역전파 변화도 초기화
      # .backward() 호출 시, 변화도 버퍼에 데이터가 계속 누적한 것을 초기화
      optimizer.zero_grad()

      # H(X) 계산: forward 연산
      hypothesis = model(input_features)
      # 비용 계산
      cost = loss_func(hypothesis, labels)
      # 역전파 수행
      cost.backward()
      optimizer.step()
   
      # 현재 batch의 스텝 별 loss 저장
      costs.append(cost.data.item())
    
    # 에폭마다 평균 비용 출력하고 모델을 저장
    print("Average Loss= {0:f}".format(np.mean(costs)))
    torch.save(model.state_dict(), os.path.join(config["output_dir"], "epoch_{0:d}.pt".format(epoch)))
    do_test(model, train_dataloader)

In [ ]:
if(__name__=="__main__"):

    root_dir = "/gdrive/My Drive/colab/rnn/stock"
    output_dir = os.path.join(root_dir, "output")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    config = {"mode": "train",
              "model_name":"epoch_{0:d}.pt".format(10),
              "output_dir":output_dir,
              "file_name": "{0:s}/samsung-2020.csv".format(root_dir),
              "sequence_len": 3,
              "input_size": 4,
              "hidden_size": 10,
              "output_size": 1,
              "num_layers": 1,
              "batch_size": 1,
              "learn_rate": 0.1,
              "epoch": 10,
              }

    if(config["mode"] == "train"):
        train(config)
    else:
        test(config)

torch.Size([72, 3, 4])
torch.Size([72, 1])
Average Loss= 907.702039
PRED= [53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1]
GOLD= [45.4, 47.3, 55.9, 60.7, 43.0, 56.5, 59.9, 61.8, 57.9, 58.8, 50.0, 60.0, 55.4, 60.0, 45.4, 56.4, 58.6, 60.4, 60.4, 60.5, 47.8, 62.3, 59.7, 58.9, 47.8, 56.5, 55.5, 62.4, 57.4, 59.2, 50.0, 47.0, 60.8, 48.3, 61.5, 59.8, 42.5, 48.3, 56.8, 59.5, 60.8, 61.4, 56.8, 61.1, 61.3, 56.5, 60.0, 54.6, 61.8, 59.1, 59.5, 54.2, 45.6, 61.3, 48.7, 57.2, 54.2, 57.2, 54.2, 57.8, 59.5, 50.8, 59.2, 59.0, 55.8, 60.7, 56.4, 55.0, 48.9, 52.1, 55.5, 60.2]
Average Loss= 30.854506
PRED= [54.5, 54.5, 54.5, 54.5,